In [53]:
import pandas as pd
import langid as lg
import nltk
from bs4 import BeautifulSoup as bsoup
import re
import os
import nltk
from nltk.collocations import *
from itertools import chain
import itertools
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.probability import *
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
import collections

tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")

In [54]:
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [55]:
in_stopwords = open('stopwords_en.txt', 'r', encoding='utf8')
in_stopwords = in_stopwords.read()
in_stopwords_set = set(in_stopwords.split('\n'))

In [56]:
def tokenize_text(inp_text):
    text = tokenizer.tokenize(inp_text)
    inp_list = [w for w in text if w not in in_stopwords_set]
    ps = nltk.stem.porter.PorterStemmer()
    inp_list = [ps.stem(w) for w in inp_list]
    #lem = nltk.stem.wordnet.WordNetLemmatizer()
    #inp_list = [lem.lemmatize(w) for w in inp_list]
    return text

In [57]:
df['clean_text'] = df['text'].apply(lambda x: tokenize_text(x.lower().strip()))
test_df['clean_text'] = test_df['text'].apply(lambda x: tokenize_text(x.lower().strip()))

In [58]:
word_in_files = list(chain.from_iterable([set(value) for value in df['clean_text']]))
freq_word_in_files = FreqDist(word_in_files)

freq_word_in_files_dict = dict(freq_word_in_files)
con_dep_stopwords = []

for key, value in freq_word_in_files_dict.items():
    # condition to filter the context dependent stopwords and append to a list initialized above
    if value > 28156 or value < 281:
        con_dep_stopwords.append(key)

In [59]:
word_in_files1 = list(chain.from_iterable([set(value) for value in test_df['clean_text']]))
freq_word_in_files1 = FreqDist(word_in_files1)

freq_word_in_files_dict1 = dict(freq_word_in_files1)
con_dep_stopwords1 = []

for key, value in freq_word_in_files_dict1.items():
    # condition to filter the context dependent stopwords and append to a list initialized above
    if value > 7040 or value < 370:
        con_dep_stopwords1.append(key)

In [60]:
def remove_con_stopwords(inp_list):
    inp_list1 = [w for w in inp_list if w not in con_dep_stopwords]
    return inp_list1

In [61]:
df['clean_text'] = df['clean_text'].apply(lambda x: remove_con_stopwords(x))

In [62]:
def remove_con_stopwords(inp_list):
    inp_list1 = [w for w in inp_list if w not in con_dep_stopwords1]
    return inp_list1

In [63]:
test_df['clean_text'] = test_df['clean_text'].apply(lambda x: remove_con_stopwords(x))

In [64]:
doc = []
for i in df['clean_text']:
    i = ' '.join(i)
    doc.append(i)

In [65]:
doc1 = []
for i in test_df['clean_text']:
    i = ' '.join(i)
    doc1.append(i)

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(doc)
X_train_counts.shape

(7613, 33)

In [67]:
count_vect.vocabulary_.get(u'algorithm')

In [68]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(7613, 33)

In [69]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(7613, 33)

In [70]:
X_new_counts = count_vect.transform(doc1)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [71]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train_tfidf, df.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [72]:
predicted = classifier.predict(X_new_tfidf)

In [73]:
from sklearn.linear_model import SGDClassifier
clf1 = SGDClassifier(max_iter=1000, tol=1e-3)

In [74]:
clf1.fit(X_train_tfidf, df.target)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [75]:
pred = clf1.predict(X_new_tfidf)

In [76]:
del test_df['keyword']
del test_df['location']
del test_df['text']
del test_df['clean_text']
test_df['target'] = pred
test_df.to_csv("nav.csv", index=False)